In [3]:
## Importar o pandas e ler o arquivo
import pandas as pd

In [4]:
df = pd.read_csv('/content/loja_livro_filmes.csv')

In [5]:
## Preparar os dados
# Agrupar a soma dos custos dos pedidos por categoria
df_produto_custo = df.groupby(['categoria'])[['custo_produto']].sum()
# Ajustar o nome da coluna e index
df_produto_custo.index.name = 'Categoria produto'
df_produto_custo.columns = ['Custo de pedidos']
df_produto_custo

## Criar a visualização
s_produto = df_produto_custo.style
# Destacar elementos
s_produto.format('R$ {:,.2f}')\
         .highlight_max(color='#F16165')\
         .highlight_min(color='green')

,Custo de pedidos
Categoria produto,
BlueRay,"R$ 14,191.36"
CD,"R$ 10,514.28"
Coleção,"R$ 73,128.59"
Livro,"R$ 78,213.80"


In [6]:
## Preparar os dados
# Tornar 'data_chegada' e 'data_pedido' colunas datetime
df['data_chegada'] = pd.to_datetime(df['data_chegada'])
df['data_pedido'] = pd.to_datetime(df['data_pedido'])

## Construir cópia de um DataFrame para adicionar novas colunas de dados
df_datas = df.copy()
df_datas = df_datas.sort_values('data_pedido')
df_datas = df_datas.reset_index(drop=True)

# Coletar apenas informações a partir do ano de 2013
df_datas = df_datas[df_datas['data_pedido'] >= '2013-01-01']

# Adicionar coluna com os meses formatados
df_datas['meses'] = df_datas['data_pedido'].dt.strftime('%Y - %b')

# Adicionar uma coluna com o tempo de entrega entre pedido e chegada
tempo_demora = (df_datas['data_chegada'] - df_datas['data_pedido']).dt.days
df_datas['tempo_entrega'] = tempo_demora

# Criar a estrutura em DataFrame
entregas = df_datas.pivot_table(index = 'pais', columns = 'meses', values = 'tempo_entrega' , aggfunc = 'mean', sort=False)

## Criar a visualização
s_entregas = entregas.style.format('{:,.2f}')
s_entregas.set_sticky(axis="index")

meses,2013 - Jan,2013 - Feb,2013 - Mar,2013 - Apr,2013 - May,2013 - Jun,2013 - Jul,2013 - Aug,2013 - Sep,2013 - Oct,2013 - Nov,2013 - Dec,2014 - Jan,2014 - Feb,2014 - Mar,2014 - Apr,2014 - May,2014 - Jun,2014 - Jul,2014 - Aug,2014 - Sep,2014 - Oct,2014 - Nov,2014 - Dec,2015 - Jan,2015 - Feb,2015 - Mar,2015 - Apr,2015 - May,2015 - Jun,2015 - Jul,2015 - Aug,2015 - Sep,2015 - Oct,2015 - Nov,2015 - Dec
pais,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Chile,90.21,341.24,470.11,351.70,195.55,99.73,307.23,207.92,321.06,137.70,80.67,285.69,633.85,483.75,658.22,252.60,309.82,469.79,149.89,358.42,403.83,341.12,327.12,329.43,530.29,549.07,606.28,424.06,423.43,353.83,432.64,447.47,314.11,502.29,406.92,254.91
Uruguai,180.88,201.27,146.52,251.06,12.50,555.25,217.79,161.82,117.33,90.00,222.76,249.60,494.30,420.64,513.40,343.81,558.94,555.58,379.56,487.89,751.57,542.15,545.12,624.85,313.26,388.35,231.84,395.70,502.35,360.04,349.48,280.59,346.00,453.33,434.12,184.42
Venezuela,216.05,267.50,219.40,221.54,180.88,315.77,282.42,393.40,405.16,187.14,370.05,264.86,550.70,529.82,447.92,713.86,735.25,441.38,336.91,541.44,183.55,441.33,279.88,336.89,510.44,739.95,307.56,619.32,413.48,442.91,443.09,236.68,258.38,327.42,271.11,187.75
Argentina,321.91,314.21,356.00,481.17,346.42,309.32,351.89,171.75,342.20,149.39,513.63,437.70,419.76,527.11,482.53,304.42,536.00,592.68,570.82,465.56,452.94,385.65,398.42,373.05,447.40,465.42,324.67,467.70,264.59,529.89,374.68,506.90,437.33,299.18,317.96,293.12
Equador,718.81,16.93,80.67,206.21,178.38,233.00,317.88,252.24,205.38,479.15,251.87,249.20,588.10,500.07,526.46,402.00,227.65,263.50,275.65,424.53,306.11,261.06,479.41,370.35,365.41,485.79,338.86,277.61,440.35,566.53,279.64,425.28,337.30,238.42,335.05,151.00
Colômbia,339.46,293.07,328.85,136.73,580.14,211.05,377.93,643.80,139.30,570.00,16.00,432.50,462.00,372.76,608.53,562.21,471.77,621.11,529.55,582.05,513.67,341.79,441.25,650.36,305.96,204.57,452.52,328.94,433.17,438.86,399.00,433.91,435.48,462.63,149.95,393.78
Brasil,238.74,520.55,406.64,382.64,454.87,232.25,181.22,266.13,272.46,213.67,279.78,241.94,532.88,652.61,328.92,365.77,479.71,291.75,430.80,454.21,534.81,434.72,416.86,144.00,284.03,380.69,431.16,432.07,375.54,222.56,310.94,455.69,392.79,394.18,391.48,241.50
Paraguai,216.86,384.73,495.88,272.29,345.21,263.57,120.42,285.29,506.47,351.67,81.72,375.62,718.06,436.29,461.08,473.91,665.82,369.50,332.13,304.64,506.08,596.90,387.39,237.08,314.32,613.50,510.65,372.72,463.72,354.54,607.22,405.35,261.52,354.65,203.30,107.25
Peru,283.48,379.05,436.15,193.47,167.53,216.08,333.33,371.07,280.93,319.38,290.62,218.32,311.59,440.88,493.32,432.79,631.00,398.39,493.86,390.50,598.83,336.21,464.33,175.80,725.22,388.39,470.48,393.83,499.64,332.81,330.58,556.28,471.40,204.71,325.76,323.09


A empresa que estamos trabalhando distribui produtos por toda América do Sul para várias categorias de clientes. Nosso desafio envolve uma visualização que permita separar os gastos de envio de cada produto (custo total) por tipo de cliente, de modo que seja possível entender o tipo de produto que mais gerou custos de envio em cada tipo de cliente. Além disso, é interessante também mostrar a distribuição desses custos por país enviado, permitindo entender também os gastos por país.

In [7]:
## Importar o pandas e ler o arquivo
import pandas as pd
df = pd.read_csv('loja_livro_filmes.csv')

## Preparar os dados

# Criar um DataFrame alternativo para ter um nome de coluna coerente com a visualização
df_alteracao = df.rename(columns={'pais': 'Custo por país ($)'})

# Agrupar os dados de tipo de cliente, com a categoria de produtos e os países que esses clientes importam o produto
df_clientes = df_alteracao.pivot_table(index=['tipo_cliente', 'categoria'], columns = 'Custo por país ($)', values = 'custo_produto', aggfunc = 'sum')

# Renomear o cabeçalho para nomes mais coerentes
df_clientes.rename_axis(index={'tipo_cliente': 'Tipo cliente', 'categoria': 'Categoria produto'}, inplace = True)

# Adicionar uma coluna que mostra a soma dos custos por tipo de cliente e categoria de produto
df_clientes['Total ($)'] = df_clientes.sum(axis=1)

## Criar a visualização
s_clientes = df_clientes.style.format('{:,.2f}')

# Criar os estilos base para a tabela, a partir da padronização das fontes, plano de fundo para branco e o alinhamento do cabeçalho para o topo
tabela = {
    'selector': 'td, th',
    'props': 'font-weight: normal; font-family: Arial; text-align: right; background-color: white;'
}
# Alinhar verticalmente o cabeçalho para o topo permite a pessoa observadora identificar onde começa a hierarquização dos grupos B2B e B2C
cabecalho = {
    'selector': 'th',
    'props': 'vertical-align: top'
}
s_clientes.set_table_styles([tabela, cabecalho])

# Adicionar linha de separação do cabeçalho superior
# Destacar as categorias que mais geraram custo com a aplicação de negrito nas fontes
s_clientes.set_table_styles({
    ('B2B', 'BlueRay'): [{'selector': 'th', 'props': 'border-top: 1px solid #181818'},
              {'selector': 'td', 'props': 'border-top: 1px solid #181818'}],
    ('B2B', 'Livro'): [{'selector': 'th', 'props': 'font-weight: bold'}],
    ('B2C', 'Livro'): [{'selector': 'th', 'props': 'font-weight: bold'}],
}, overwrite=False, axis=1)

# Destacar os elementos em cada tipo de cliente que mais geraram custos com uma seleção
s_clientes.set_table_styles({
    'Total ($)': [{
        'selector': '.true',
        'props': 'font-weight: bold'
    }]
},overwrite=False,axis=0)

cores_coluna = pd.DataFrame(['false','false','false','true','false','false','false','true'],index= df_clientes['Total ($)'].index,
                            columns = ['Total ($)'])
# Visualização final
s_clientes.set_td_classes(cores_coluna)